In [ ]:
import pandas as pd
import tabula
from Levenshtein import distance
import numpy as np
import scipy

In [91]:
class Formateador:
    def __init__(self):
        pass

    def relacionar_sigfe_y_sigcom(self):
        df_bodega_sigfe = pd.read_excel('asociacion_bodega_sigfe.xlsx', header = 3)
        df_sigfe_sigcom = pd.read_excel('asociacion_sigfe_sigcom.xlsx')

        df_bodega_sigfe_unicos = df_bodega_sigfe.drop_duplicates(subset = 'Código')
        df_bodega_sigfe_unicos['Items'] = df_bodega_sigfe_unicos['Items'].apply(self.convertidor_int_a_formato_sigfe)
        df_bodega_sigfe_unicos['Cod. SIGFE limpio'] = df_bodega_sigfe_unicos['Items'].apply(self.formato_chequeador)
        df_bodega_sigfe_unicos = df_bodega_sigfe_unicos.rename(columns = {'Items': 'Cod. SIGFE', 'Código': 'Cod. Bodega'})

        left_df = df_bodega_sigfe_unicos.iloc[:, [0, 1, 2, 3, 4, 5, -1]]
        right_df = df_sigfe_sigcom

        df_adquisiciones_sigcom = pd.merge(left_df, right_df, how = 'left', on = 'Cod. SIGFE limpio')
        df_adquisiciones_sigcom = df_adquisiciones_sigcom.drop(columns = 'Cod. SIGFE limpio')
        df_adquisiciones_sigcom = df_adquisiciones_sigcom.set_index('Cod. Bodega', drop = True)
        df_adquisiciones_sigcom = df_adquisiciones_sigcom[['Descripción', 'Cod. SIGFE_x', 'Cod. SIGCOM', 'Nº', 'Familia', 'Nombre Items', 'Item en SIGCOM', 'Item SIGFE', 'Gastos SIGCOM']]
        df_adquisiciones_sigcom.rename(columns = {'Cod. SIGFE_x': 'Cod. SIGFE'}, inplace = True)

        df_adquisiciones_sigcom.to_json('asociacion_bodega_sigcom.json', orient = 'index', indent = 1, force_ascii = False)
        df_adquisiciones_sigcom.to_excel('asociacion_bodega_sigcom.xlsx')
    
    def convertidor_int_a_formato_sigfe(self, numero):
        numero = str(numero)
        primer_bloque = numero[0: 2]
        segundo_bloque = numero[2: 4]
        tercer_bloque = numero[4: 7]

        nuevo_formato = '.'.join([primer_bloque, segundo_bloque, tercer_bloque])

        if len(numero) >= 10:
            cuarto_bloque = numero[7: 10]
            nuevo_formato = '.'.join([nuevo_formato, cuarto_bloque])

            if len(numero) == 12:
                quinto_bloque = numero[10: 12]
                nuevo_formato = '.'.join([nuevo_formato, quinto_bloque])
        
        return nuevo_formato
    
    def formato_chequeador(self, numero_formateado):
        excepciones = ['22.01.001', '22.02.002', '22.04.004.001', '22.04.005', '22.06.002', '22.09.005',
                    '22.11.002', '22.12.999.009']
        
        for excepcion in excepciones:
            if excepcion in numero_formateado:
                numero_formateado = excepcion
                break
        
        return numero_formateado
    
    def obtener_descripciones_repetidas(self):
        df = pd.read_excel('asociacion_bodega_sigcom.xlsx')
        df['Cod. Bodega'] = df['Cod. Bodega'].astype(str).apply(lambda x: x.strip())
        descr_repetidas = df[df.duplicated('Descripción', keep = False)].sort_values(by = 'Descripción')
        agrupados = descr_repetidas.groupby(['Descripción'])['Cod. Bodega'].value_counts()
        agrupados.to_excel('descripciones_con_mas_de_un_cod_bodega.xlsx')

        return agrupados


In [80]:
formateador = Formateador()
formateador.relacionar_sigfe_y_sigcom()
repetidas = formateador.obtener_descripciones_repetidas()

C:\Users\gmolina\AppData\Local\Temp\ipykernel_25248\57672787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bodega_sigfe_unicos['Items'] = df_bodega_sigfe_unicos['Items'].apply(self.convertidor_int_a_formato_sigfe)
C:\Users\gmolina\AppData\Local\Temp\ipykernel_25248\57672787.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bodega_sigfe_unicos['Cod. SIGFE limpio'] = df_bodega_sigfe_unicos['Items'].apply(self.formato_chequeador)


In [122]:
Y = scipy.spatial.distance.pdist(np.reshape(list(df['Cod. Bodega']), (-1, 1)), distance)
Y = scipy.spatial.distance.squareform(Y)

In [124]:
pd.DataFrame(Y)

,0,1,2,3,4,5,6,7,8,9,...,9419,9420,9421,9422,9423,9424,9425,9426,9427,9428
0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9424,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
9425,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
9426,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
9427,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
